# Chương 8: Machine Learning
Chương này yêu cầu bạn thực hiện bài toán sentiment analysis trên corpus sentence polarity dataset v1.0 trong Moview Review Data của tác giả Bo Pang và Lillian Lee. Yêu cầu của bài toán sentiment analysis là phân loại các câu thành positive và negative sentiments.

# 70. Download và tiền xử lý dữ liệu
Sử dụng dữ liệu liên quan đến sentiment polarity của các câu (download tại đây), tạo dữ liệu chuẩn hoá (sentiment.txt) theo hướng dẫn dưới đây.

1. Thêm vào '+1' ở bắt đầu các dòng trong file rt-polarity.pos (giữa +1 và nội dung của câu cách nhau bởi ký tự trắng).

2. Thêm vào '-1' ở bắt đầu các dòng trong file rt-polarity.neg (giữa -1 và nội dung của câu cách nhau bởi ký tự trắng).

3. Kết hợp nội dung thu được trong phần 1 và 2 để tạo thành file sentiment.txt

Sau khi đã thu được file sentiment.txt, xác nhận số lượng các câu với positive polarity và các câu với negative polarity.

In [2]:
# Read files
with open('rt-polaritydata/rt-polaritydata/rt-polarity.pos', 'r') as f:
    pos = f.readlines()
    
with open('rt-polaritydata/rt-polaritydata/rt-polarity.neg', 'r') as f:
    neg = f.readlines()

In [6]:
# Create sentiment.txt
with open('sentiment.txt', 'w') as f:
    for line in pos:
        f.write('+1 ' + line)
    
    for line in neg:
        f.write('-1 ' + line)

In [7]:
# Check sentiment.txt
with open('sentiment.txt', 'r') as f:
    sentiment_data = f.readlines()
    
print(len(sentiment_data))
print('Pos sentence: ' + sentiment_data[0])
print('Neg sentence: ' + sentiment_data[-1])

10662
Pos sentence: +1 the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . 

Neg sentence: -1 enigma is well-made , but it's just too dry and too placid . 

